In [1]:
import os
import pandas as pd
import sqlite3
from docx import Document
import sys
from io import BytesIO

from pretraitement import Pretraitement
from database import database
from similarity import Similarity
import json
from sentence_transformers import SentenceTransformer, losses,InputExample

/Users/user/Desktop/Stage2A/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-08 15:19:48.343312: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
conn=database("data.db")
conn.connect()



### Préparation des données

In [3]:
df = conn.readQuestionnaire()

In [4]:
CahierCharge="Cahier des charges_nouveau poste 2025_V0.6_définitive.pdf"

CahierCharge = conn.readCahierDeCharges(CahierCharge)


CahierCharge

"Version : mai 2025 \n \n \nCAHIER DES CHARGES \n \n \n \nVersion révisée du : 27.05.2025 \n \n \nAnnule et remplace la version du :       \n \n1. Identification du poste \nDicastère : \nCCRH \nService : \nService des ressources humaines (SRH) \nDénomination du poste : \nChargé-e de projets RH et responsable de la formation continue \n2. Positionnement hiérarchique du poste \n2.1. Dénomination du poste du/de la supérieur-e direct-e : \nChef-fe du Service des ressources humaines \n2.2. Postes hiérarchiquement subordonnés : \n-- \n3. Titulaire \nNom :       \nPrénom :       \nTaux d’activité : 70 % \n \n4. Mode de remplacement \n4.1. Le/la titulaire remplace : \nLe/la spécialiste de la sécurité au travail et de la protection de la santé / \n \nLe/la partenaire RH, responsable de la formation des apprenant-e-s  \n \ndans les domaines suivants : \nFormations dans le domaine de la santé,  \n \n \n \n \nFormation des apprenant-e-s \n4.2. Le/la titulaire est remplacé-e par : \nLes partenaires

In [5]:
questionnaire = df


preproc = Pretraitement(CahierCharge, questionnaire)
preproc._load_spacy()
df= preproc.load_questionnaire()
df=preproc.keep_abcd_lines(df)

sentences = preproc.build_cahier_df()
df

,classe,title,response,pts
0,A,Formation de base,La fonction requiert au minimum le niveau scol...,6.37000
1,A,Formation de base,La fonction requiert une formation professionn...,8.64500
2,A,Formation de base,La fonction requiert un certificat de fin de s...,11.37500
3,A,Formation de base,La fonction requiert un certificat de fin de s...,15.92500
4,A,Formation de base,La fonction requiert un Bachelor (Université o...,20.93000
...,...,...,...,...
86,D,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,65.78000
87,D,Innovation,L'exercice de la fonction s'effectue dans le c...,23.16600
88,D,Innovation,L'exercice de la fonction requiert la capacité...,34.74900
89,D,Innovation,L'exercice de la fonction requiert des compéte...,50.19300


In [6]:
sentences

,sentence
0,"Raison d'être (Mission)\nDans le quotidien, co..."
1,Formation et expériences professionnelles requ...
2,Aucune formation complémentaire n'est nécessai...
3,La durée de l'expérience professionnelle est d...
4,Responsable hiérarchique direct\nResponsable d...
5,"Collaborer à la santé et aux soins corporels, ..."
6,"Collaborer à la santé et aux soins corporels, ..."
7,Encadrer et accompagner le bénéficiaire de pre...
8,Respecter les prescriptions d'hygiène et de sé...
9,Apporter son assistance dans les activités d'e...


### similarité entre les paragraphes et les reponses

In [7]:
from similarity import Similarity

In [8]:
sim       = Similarity(batch_size=32)      
matches   = sim.top_k_matches(
    questions=df["response"].tolist(),
    corpus_sentences=sentences["sentence"].tolist(),
    k=5,
    question_titles=df["title"].tolist(),
)


matches.to_excel("results/Test5Results.xlsx", index=False)
matches

,question_title,question,sentence,score,rank
0,Formation de base,La fonction requiert au minimum le niveau scol...,Aucune formation complémentaire n'est nécessai...,0.544583,1
1,Formation de base,La fonction requiert au minimum le niveau scol...,Formation et expériences professionnelles requ...,0.499906,2
2,Formation de base,La fonction requiert au minimum le niveau scol...,La durée de l'expérience professionnelle est d...,0.343500,3
3,Formation de base,La fonction requiert au minimum le niveau scol...,SMQ\nVeiller à l'application et au respect du ...,0.272818,4
4,Formation de base,La fonction requiert au minimum le niveau scol...,Veille informative\n8.1.,0.265545,5
...,...,...,...,...,...
450,Innovation,L'exercice de la fonction requiert des compéte...,Autres activités et responsabilités\n(selon cr...,0.555119,1
451,Innovation,L'exercice de la fonction requiert des compéte...,Innovation\nContribuer par des remarques motiv...,0.471019,2
452,Innovation,L'exercice de la fonction requiert des compéte...,"Raison d'être (Mission)\nDans le quotidien, co...",0.470332,3
453,Innovation,L'exercice de la fonction requiert des compéte...,"Participer aux tâches d'administration, de log...",0.460816,4


In [23]:
print(len(matches), "matches found")
matches_no_duplicates = matches.drop_duplicates(subset=["question_title", "sentence"])


455 matches found


In [24]:
print(len(matches_no_duplicates))

# Drop columns only if they exist in the DataFrame
cols_to_drop = [col for col in ["score", "rank", "label"] if col in matches_no_duplicates.columns]
if cols_to_drop:
    matches_no_duplicates.drop(columns=cols_to_drop, inplace=True)
matches_no_duplicates

170


/var/folders/hw/687qrsn947v91l2yv331ngdc0000gn/T/ipykernel_16983/1285617893.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_no_duplicates.drop(columns=cols_to_drop, inplace=True)


,question_title,question,sentence
0,Formation de base,La fonction requiert au minimum le niveau scol...,Aucune formation complémentaire n'est nécessai...
1,Formation de base,La fonction requiert au minimum le niveau scol...,Formation et expériences professionnelles requ...
2,Formation de base,La fonction requiert au minimum le niveau scol...,La durée de l'expérience professionnelle est d...
3,Formation de base,La fonction requiert au minimum le niveau scol...,SMQ\nVeiller à l'application et au respect du ...
4,Formation de base,La fonction requiert au minimum le niveau scol...,Veille informative\n8.1.
...,...,...,...
436,Innovation,L'exercice de la fonction s'effectue dans le c...,Autres activités et responsabilités\n(selon cr...
437,Innovation,L'exercice de la fonction s'effectue dans le c...,"Accomplir sa mission, en lien avec ""la politiq..."
438,Innovation,L'exercice de la fonction s'effectue dans le c...,Rester attentif/ve à l'évolution de l'accompag...
439,Innovation,L'exercice de la fonction s'effectue dans le c...,Innovation\nContribuer par des remarques motiv...


In [25]:
mapping={}
for i, row in matches_no_duplicates.iterrows():
    question_title = row["question_title"]
    sentence = row["sentence"]
    if question_title not in mapping:
        mapping[question_title] = []
    mapping[question_title].append(sentence)

mapping


{'Formation de base': ["Aucune formation complémentaire n'est nécessaire pour l'exercice de la fonction.",
  'Formation et expériences professionnelles requises\n(Formation de base, formation complémentaire, expériences)\nLa fonction requiert une formation professionnelle initiale de deux ans (AFP-ASA) ou équivalent.',
  "La durée de l'expérience professionnelle est déterminée dans le cadre du processus de recrutement (de débutant accepté/e à expérimenté/e).",
  "SMQ\nVeiller à l'application et au respect du système de gestion qualité.",
  'Veille informative\n8.1.',
  "Autres activités et responsabilités\n(selon critères d'evaluation des fonctions des etablissements de l'anmea)\nau niveau :\nRelations internes et externes\nDéveloppement personnel\nVeiller de sa propre initiative à l'adéquation de ses compétences avec les exigences et besoins évolutifs du poste et mettre en œuvre les formations continues et autres démarches nécessaires.",
  "Faire preuve de la réserve et de la discréti

In [26]:
rows = []
for title, sentences in mapping.items():
    for sentence in sentences:
        rows.append({'title': title, 'sentence': sentence})
df_sentencized = pd.DataFrame(rows)
df_sentencized

,title,sentence
0,Formation de base,Aucune formation complémentaire n'est nécessai...
1,Formation de base,Formation et expériences professionnelles requ...
2,Formation de base,La durée de l'expérience professionnelle est d...
3,Formation de base,SMQ\nVeiller à l'application et au respect du ...
4,Formation de base,Veille informative\n8.1.
...,...,...
165,Innovation,Autres activités et responsabilités\n(selon cr...
166,Innovation,"Accomplir sa mission, en lien avec ""la politiq..."
167,Innovation,Rester attentif/ve à l'évolution de l'accompag...
168,Innovation,Innovation\nContribuer par des remarques motiv...


### sentencization plus détailée --> phrases courtes

In [27]:
mappingSentencized = {}
for question_title, sentences_list in mapping.items():
    sentencized = preproc.sentencize_sentences(sentences_list)
    mappingSentencized[question_title] = sentencized

mappingSentencized

{'Formation de base': ["Aucune formation complémentaire n'est nécessaire pour l'exercice de la fonction",
  'Formation et expériences professionnelles requises',
  'Formation de base',
  'La fonction requiert une formation professionnelle initiale de deux ans',
  "La durée de l'expérience professionnelle est déterminée dans le cadre du processus de recrutement",
  'de débutant accepté/e à expérimenté/e',
  "Veiller à l'application et au respect du système de gestion qualité",
  'Autres activités et responsabilités',
  "selon critères d'evaluation des fonctions des etablissements de l'anmea",
  'Relations internes et externes',
  "Veiller de sa propre initiative à l'adéquation de ses compétences avec les exigences et besoins évolutifs du poste et mettre en œuvre les formations continues et autres démarches nécessaires",
  "Faire preuve de la réserve et de la discrétion qu'exige sa fonction",
  'Collaborer à la santé et aux soins corporels',
  "en secteur de soins et en secteur d'animati

In [28]:
rows = []
for title, sentences in mappingSentencized.items():
    for sentence in sentences:
        rows.append({'title': title, 'sentence': sentence})
df_sentencized = pd.DataFrame(rows)
df_sentencized

,title,sentence
0,Formation de base,Aucune formation complémentaire n'est nécessai...
1,Formation de base,Formation et expériences professionnelles requ...
2,Formation de base,Formation de base
3,Formation de base,La fonction requiert une formation professionn...
4,Formation de base,La durée de l'expérience professionnelle est d...
...,...,...
715,Innovation,du fonctionnement du/des lieu/x de vie géré/s ...
716,Innovation,Participer aux tâches d'administration
717,Innovation,de logistique et d'organisation du travail
718,Innovation,1 Consigner tout changement observé et transme...


In [ ]:
def contains_verb(text):
    doc = nlp(text)
    return any(token.pos_ == "VERB" or token.pos_ == "AUX" for token in doc)


In [56]:
import spacy

nlp = spacy.load("fr_core_news_md")

df_sentencized = df_sentencized[df_sentencized["sentence"].apply(contains_verb)]
df_without_verbs = df_sentencized[~df_sentencized["sentence"].apply(contains_verb)]
for i,row in df_sentencized.iterrows():
    
    print(row['sentence'])
    

Aucune formation complémentaire n'est nécessaire pour l'exercice de la fonction
La fonction requiert une formation professionnelle initiale de deux ans
La durée de l'expérience professionnelle est déterminée dans le cadre du processus de recrutement
de débutant accepté/e à expérimenté/e
Veiller à l'application et au respect du système de gestion qualité
Veiller de sa propre initiative à l'adéquation de ses compétences avec les exigences et besoins évolutifs du poste et mettre en œuvre les formations continues et autres démarches nécessaires
Faire preuve de la réserve et de la discrétion qu'exige sa fonction
Collaborer à la santé et aux soins corporels
1 Accompagner le bénéficiaire de prestations aux toilettes
2 Aider le bénéficiaire de prestations à s'alimenter
3 Aider le bénéficiaire de prestations à s'habiller et à se déshabiller
4 Participer à l'accompagnement de personnes en fin de vie
Assurer la circulation de l'information en veillant à la qualité
Collaborer à la santé et aux soi

In [57]:
for i,row in df_without_verbs.iterrows():
    
    print(row['sentence'])
    

In [58]:
# sim       = Similarity(batch_size=32)      
# matchesSentences   = sim.top_k_matches(
#     questions=df["response"].tolist(),
#     corpus_sentences=df_sentencized["sentence"].tolist(),
#     k=5,
#     question_titles=df["title"].tolist(),
# )
# # 3. Export ou annotation
# matchesSentences.to_excel("Test5Resultslatest.xlsx", index=False)
# print("✅  Fichier 'Test5Resultslatest.xlsx' généré !")


### Similarités entre les phrases courtes de chaque classe et les nivs de réps

In [59]:
sim       = Similarity(batch_size=32)   
titles = [
    "Formation de base",
     "Formation complémentaire",
        "Nature",
            "Durée",
                "Autonomie de décision",
    "Analyse et synthèse",

    "Complexité de l'environnement",
    "Connaissances linguistiques",
    "Diversité des missions",
    "Diversité et quantité des postes à gérer",

    "Evolution de l'environnement",
   
    
    "Impact externe des prestations",
    "Impact interne des prestations",
    "Innovation",
 
    "Nature des communications externes",
    "Nature des communications internes",
    "Responsabilités budgétaires",
    "Responsabilités de planification et de réalisation des activités à court terme",
    "Responsabilités de planification et de réalisation des activités à long terme",
    "Rôle dans la gestion des ressources humaines"
]
all_matches = []
for title in titles:
    questions = df[df["title"] == title]["response"].tolist()
    corpus_sentences = mappingSentencized.get(title, [])
    if questions and corpus_sentences:
        matches = sim.top_k_matches(
            questions=questions,
            corpus_sentences=corpus_sentences,
            k=5,
            question_titles=[title]*len(questions),
        )
        all_matches.append(matches)

matchesSentences = pd.concat(all_matches, ignore_index=True)
matchesSentences.to_excel("results/Test5Resultslatest.xlsx", index=False)
print("✅  Fichier 'Test5Resultslatest.xlsx' généré !")



✅  Fichier 'Test5Resultslatest.xlsx' généré !


### Orgganisation des résultats pour afficher les meilleures réps

In [60]:
import json


sentences_used=[]
results_for_json = []
for title, group in matchesSentences.groupby('question_title'):
    max_score = group['score'].max()
    threshold = 0.03
    close_matches = group[(group['score'] >= max_score - threshold) & (group['score'] <= max_score + threshold)].sort_values('score', ascending=False)
    close_matches = close_matches.drop_duplicates(subset=["sentence"])
    print(f"\n--- {title} (top score: {max_score:.3f}) ---")
    


    for _, row in close_matches.iterrows():
        if row['sentence'] not in sentences_used:
            sentences_used.append(row['sentence'])
        pts_value = df.loc[df["response"] == row["question"], "pts"]

        print(f"Score: {row['score']:.3f}\nQuestion: {row['question']}\nSentence: {row['sentence']}\n")
        print(f"pts: {pts_value.values[0]}")
        print("-" * 80)

        results_for_json.append({
            "title": title,
            "question": row["question"],
            "sentence": row["sentence"],
            "score": row["score"],
            "pts": pts_value.values[0] 
        })



if title == titles[-1]: 
    with open("results/all_matches.json", "w", encoding="utf-8") as f:
        json.dump(results_for_json, f, ensure_ascii=False, indent=2)
    json_content_str = json.dumps(results_for_json, ensure_ascii=False)
    conn.execute_query('''delete from ResultatsJSON where filename = ?''', ("all_matches.json",))
    conn.execute_query('''
    INSERT INTO ResultatsJSON (filename, json_content) VALUES (?, ?)
''', ("all_matches.json", json_content_str))
    conn.commit()



--- Analyse et synthèse (top score: 0.519) ---
Score: 0.519
Question: La fonction requiert la faculté d'analyser des sujets parfois complexes en se référant à des modèles et à des connaissances déjà expérimentés.
Sentence: gérer d'une manière professionnelle des situations définies et structurées

pts: 29.601
--------------------------------------------------------------------------------

--- Autonomie de décision (top score: 0.791) ---
Score: 0.791
Question: La fonction applique des consignes et des directives établies.
Sentence: en appliquant des consignes

pts: 23.4
--------------------------------------------------------------------------------

--- Complexité de l'environnement (top score: 0.503) ---
Score: 0.503
Question: La fonction fait fréquemment face à un degré important de difficultés, souvent nouvelles, et les situations générées requièrent adaptabilité, flexibilité et initiative.
Sentence: Veiller de sa propre initiative à l'adéquation de ses compétences avec les exigen

### Phrases not used

In [61]:

sentences = []
for i in mappingSentencized:
    for j in mappingSentencized[i]:
        if j not in sentences:
            sentences.append(j)
print("Total unique sentences:", len(sentences))
print("\n",sentences)
print("Total sentences used:", len(sentences_used))
print("\n",sentences_used)

Total unique sentences: 91

 ["Aucune formation complémentaire n'est nécessaire pour l'exercice de la fonction", 'Formation et expériences professionnelles requises', 'Formation de base', 'La fonction requiert une formation professionnelle initiale de deux ans', "La durée de l'expérience professionnelle est déterminée dans le cadre du processus de recrutement", 'de débutant accepté/e à expérimenté/e', "Veiller à l'application et au respect du système de gestion qualité", 'Autres activités et responsabilités', "selon critères d'evaluation des fonctions des etablissements de l'anmea", 'Relations internes et externes', "Veiller de sa propre initiative à l'adéquation de ses compétences avec les exigences et besoins évolutifs du poste et mettre en œuvre les formations continues et autres démarches nécessaires", "Faire preuve de la réserve et de la discrétion qu'exige sa fonction", 'Collaborer à la santé et aux soins corporels', "en secteur de soins et en secteur d'animation", '1 Accompagner

In [62]:
to_use = [s for s in sentences if s not in sentences_used]
print("Total sentences not used:", len(to_use))
for s in to_use:
    if len(s) > 2:
        print(s)

Total sentences not used: 78
Formation de base
La durée de l'expérience professionnelle est déterminée dans le cadre du processus de recrutement
de débutant accepté/e à expérimenté/e
Veiller à l'application et au respect du système de gestion qualité
selon critères d'evaluation des fonctions des etablissements de l'anmea
Relations internes et externes
Faire preuve de la réserve et de la discrétion qu'exige sa fonction
Collaborer à la santé et aux soins corporels
en secteur de soins et en secteur d'animation
1 Accompagner le bénéficiaire de prestations aux toilettes
2 Aider le bénéficiaire de prestations à s'alimenter
3 Aider le bénéficiaire de prestations à s'habiller et à se déshabiller
4 Participer à l'accompagnement de personnes en fin de vie
Assurer la circulation de l'information en veillant à la qualité
la fiabilité et la traçabilité de cette dernière
en secteur de soins
1 Effectuer des soins corporels aux bénéficiaires de prestations
coucher et repositionner les bénéficiaires de

In [63]:
with open("results/not_matched.json", "w", encoding="utf-8") as f:
    json.dump(to_use, f, ensure_ascii=False, indent=2)
json_content_str = json.dumps(to_use, ensure_ascii=False)

conn.execute_query('''delete from ResultatsJSON where filename = ?''', ("not_matched.json",))

conn.execute_query('''
    INSERT INTO ResultatsJSON (filename, json_content) VALUES (?, ?)
''', ("not_matched.json", json_content_str))

conn.commit()

